In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterstats import zonal_stats
import os
import numpy as np
import momepy
import networkx as nx
import fiona
from shapely.geometry import Point, LineString
import richdem as rd
import rasterio.features
from scipy.ndimage import distance_transform_edt
import pandas as pd

c:\Users\andre\anaconda3\envs\tama\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\andre\anaconda3\envs\tama\lib\site-packages\richdem\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# 1. Parameters

In [ ]:
CUT_PATH = r".\Cut_data"
GRAPH_PATH = r".\Raw_graph"
FEATURES_PATH = r".\Raw_Features"
FINAL_GRAPH_PATH = r".\Final_Graph"

FEATURE_FILES = [
    os.path.join(FEATURES_PATH, "feature_twi.tif"),
    os.path.join(FEATURES_PATH, "feature_slope.tif"),
    os.path.join(FEATURES_PATH, "feature_distance_to_combined_river.tif")
]


DTM_PATH = os.path.join(CUT_PATH, "dtm.tif")
ALAG_PATH = os.path.join(CUT_PATH, "Alagamentos_2019.shp")
RIO_PATH = os.path.join(CUT_PATH, "tamanduatei.shp")
AF_PATH = os.path.join(CUT_PATH, "afluentes.shp")
REDE_GPKG = os.path.join(GRAPH_PATH, "raw_street_graph.gpkg")

MICROBACIA_PATH = os.path.join('./Raw_data/SHP_Tamanduatei', "Microbacias_Tamanduatei.shp")

MAX_DISTANCE = 30
CRS_METRICO = 31983

# 2. Loading and Preprocessing data

## 2.1. DTM

In [ ]:
with rasterio.open(DTM_PATH) as src:
    data_np = src.read(1, out_dtype=np.float64) 
    nodata_val = src.nodata if src.nodata is not None else -999999.0
    transform_matrix = src.transform
    pixel_width_x = src.res[0]
    profile = src.profile

valid_mask = ~np.isclose(data_np, nodata_val, rtol=1e-05, atol=1e-08)
data_np[~valid_mask] = np.nan
dtm_rd = rd.rdarray(data_np.astype(np.float32), no_data=np.nan) 
dtm_rd.transform = transform_matrix 

dtm_filled = rd.FillDepressions(dtm_rd, epsilon=True)

## 2.2. Calculating Distance to River

In [ ]:
tamanduatei_gdf = gpd.read_file(RIO_PATH)
afluentes_gdf = gpd.read_file(AF_PATH)

rios_combinados_gdf = gpd.GeoDataFrame(
    pd.concat([tamanduatei_gdf, afluentes_gdf], ignore_index=True), 
    crs=tamanduatei_gdf.crs
)

with rasterio.open(DTM_PATH) as src:
    shapes = [(geom, 1) for geom in rios_combinados_gdf.geometry]
    rios_raster = rasterio.features.rasterize(
        shapes, 
        out_shape=src.shape, 
        transform=src.transform, 
        fill=0, 
        all_touched=True, 
        dtype=np.uint8
    )

dist_mask = np.where(rios_raster == 1, 0, 1) 
distance_pixels = distance_transform_edt(dist_mask)

distance_to_river = distance_pixels * pixel_width_x
distance_to_river[np.isnan(dtm_filled)] = np.nan

## 2.3. Calculating Slope, Flow Accumulation and TWI

In [ ]:
slope = rd.TerrainAttribute(dtm_filled, attrib='slope_riserun')
flow_accumulation = rd.FlowAccumulation(dtm_filled, method='Dinf')
epsilon = 0.0001
specific_catchment_area = flow_accumulation / pixel_width_x
tan_beta = slope + epsilon
twi = np.log(specific_catchment_area / tan_beta)

## 2.4. Streets Network

In [ ]:
layer_name = 'edges'

ruas_gdf = gpd.read_file(REDE_GPKG, layer=layer_name)
alagamentos_gdf = gpd.read_file(ALAG_PATH)

if ruas_gdf.crs != CRS_METRICO:
    ruas_gdf = ruas_gdf.to_crs(epsg=CRS_METRICO) 
    alagamentos_gdf = alagamentos_gdf.to_crs(epsg=CRS_METRICO)

geom_mode = ruas_gdf.geom_type.mode()[0]

print(f"Predominant Geometry in the Network: {geom_mode}")

## 2.5. Microbacins Processing

In [ ]:
microbacias_gdf = gpd.read_file(MICROBACIA_PATH)

if microbacias_gdf.crs != ruas_gdf.crs:
    microbacias_gdf = microbacias_gdf.to_crs(ruas_gdf.crs)


id_col = microbacias_gdf.columns.drop('geometry')[0] 

ruas_gdf = gpd.sjoin(
    ruas_gdf, 
    microbacias_gdf[[id_col, 'geometry']].rename(columns={id_col: 'microbacia_id'}), 
    how='left', 
    predicate='intersects'
)
ruas_gdf = ruas_gdf.drop(columns=['index_right'], errors='ignore')

ruas_gdf['microbacia_id'] = ruas_gdf['microbacia_id'].fillna('NA')

## 2.5. Saving Raw Features

In [ ]:
profile.update(dtype=rasterio.float32, nodata=np.nan) 

def save_raster(data_array, filename, profile):
    output_data = data_array.astype(rasterio.float32)
    with rasterio.open(os.path.join(FEATURES_PATH, filename), 'w', **profile) as dst:
        dst.write(output_data, 1)

save_raster(slope, "feature_slope.tif", profile)
save_raster(flow_accumulation, "feature_flow_acc.tif", profile)
save_raster(twi, "feature_twi.tif", profile)
save_raster(distance_to_river, "feature_distance_to_combined_river.tif", profile)

# 3. Feature Engineering

## 3.1. Calculating mean `twi`, `slope` and `distance_river` for each street (edges) on the graph

In [ ]:
feature_names = ['mean_twi', 'mean_slope', 'mean_dist_river']

for i, feature_file in enumerate(FEATURE_FILES):
    feature_name = feature_names[i]
    stats = zonal_stats(
        vectors=ruas_gdf.geometry, 
        raster=feature_file, 
        stats=['mean'], 
        geojson_out=False, 
        nodata=np.nan, 
        all_touched=True 
    )
    ruas_gdf[feature_name] = [s['mean'] for s in stats]

## 3.2. Mapping Features and Labels to Graph Edges

In [ ]:
ruas_gdf['safe_id'] = ruas_gdf.index
ruas_gdf = ruas_gdf.reset_index(drop=True)
ruas_gdf['safe_id'] = ruas_gdf.index 

alagamentos_gdf['is_flooded'] = 1 
alagamentos_simplificado = alagamentos_gdf[['is_flooded', 'geometry']].copy()

ruas_com_alagamento = gpd.sjoin_nearest(
    ruas_gdf, 
    alagamentos_simplificado, 
    how='left', 
    max_distance=MAX_DISTANCE
)

ruas_agrupadas = ruas_com_alagamento.groupby('safe_id').agg({'is_flooded': 'max'})
ruas_gdf = ruas_gdf.join(ruas_agrupadas, on='safe_id')
ruas_gdf['flood_label'] = ruas_gdf['is_flooded'].fillna(0).astype(int)


feature_columns = ['mean_twi', 'mean_slope', 'mean_dist_river']

ruas_gdf[feature_columns] = ruas_gdf[feature_columns].fillna(0.0)

COLUNAS_FINAIS = feature_columns + ['flood_label', 'geometry', 'microbacia_id']
ruas_gdf_final = ruas_gdf[COLUNAS_FINAIS].copy()

# 4. Building and Saving Graph

In [ ]:
G = momepy.gdf_to_nx(ruas_gdf_final, approach='primal')
print(f"NetworkX Graph created. Nodes: {G.number_of_nodes()}, Edges: {G.number_of_edges()}")

In [ ]:
for key in list(G.graph.keys()):
    if not isinstance(G.graph[key], (str, int, float, bool)):
        del G.graph[key]

for data_struct in [G.nodes(data=True), G.edges(keys=True, data=True)]:
    for item in data_struct:
        data = item[-1]
        keys_to_delete = []
        for k, val in data.items():
            if isinstance(val, (Point, LineString, type(None))) or pd.isna(val):
                    keys_to_delete.append(k)
        for k in keys_to_delete:
            del data[k]
    

In [ ]:
GRAPHML_PATH = os.path.join(FINAL_GRAPH_PATH, "ml_ready_graph.graphml")
nx.write_graphml(G, GRAPHML_PATH)

In [ ]:
GPKG_PATH = os.path.join(FINAL_GRAPH_PATH, "visualization_ready.gpkg")
ruas_gdf_final.to_file(GPKG_PATH, driver="GPKG")